# Training MobileNet models

For mobilenets v1

In [1]:
import torch

from mobilenet_v1 import DepthwiseSeparableConv

In [ ]:
x = torch.randn(1, 3, 224, 224)

c1 = torch.nn.Conv2d(3, 32, 3, stride=2, padding=1)
x = c1(x)
dsc = DepthwiseSeparableConv(32, 32)
out = dsc(x)

print(out)
print(out.size())

In [38]:
from mobilenet_v1 import MobileNetV1

model = MobileNetV1(width_multiplier=0.85)
x = torch.randn(4, 3, 224, 224)
out = model(x)
print(out.size())


torch.Size([4, 870])
